In [172]:
# Implementing particle filter for pacman tracking

In [173]:
import random

import cv2
import numpy as np

In [174]:
def manhattanweights(g1, g2, g3, g4, pc):
    w1 = 1 / (abs(g1[0] - pc[0]) + abs(g1[1] - pc[1]) + 1e-7)
    w2 = 1 / (abs(g2[0] - pc[0]) + abs(g2[1] - pc[1]) + 1e-7)
    w3 = 1 / (abs(g3[0] - pc[0]) + abs(g3[1] - pc[1]) + 1e-7)
    w4 = 1 / (abs(g4[0] - pc[0]) + abs(g4[1] - pc[1]) + 1e-7)
    return [w1, w2, w3, w4]

def motion_model_ghosts(g,pc,allow_motion):
    ad = au = ar = al = 1
    if g[0] > pc[0]:
        al += (g[0] - pc[0])
    elif g[0] < pc[0]:
        ar += (pc[0] - g[0])
    if g[1] > pc[1]:
        au += (g[1] - pc[1])
    elif g[1] < pc[1]:
        ad += (pc[1] - g[1])  


    # normalize probabilities
    sum = al + ad + au + ar
    p_l = al/sum
    p_r = ar/sum
    p_u = au/sum
    p_d = ad/sum

    # Given Dictionary of allowable motions, if cannot move direction, split the probabilities to other directions
    if allow_motion['u'] == 0:
        p_r += 0.4*p_u
        p_l += 0.4*p_u
        p_d += 0.2*p_u
        p_u = 0
    
    if allow_motion['d'] == 0:
        p_r += 0.4*p_d
        p_l += 0.4*p_d
        p_u += 0.2*p_u
        p_d = 0
    
    if allow_motion['r'] == 0:
        p_u += 0.4*p_r
        p_d += 0.4*p_r
        p_l += 0.2*p_r
        p_r = 0
    
    if allow_motion['l'] == 0:
        p_u += 0.4*p_l
        p_d += 0.4*p_l
        p_r += 0.2*p_l
        p_l = 0

    return p_l,p_r,p_u,p_d

def motion_model(g1, g2, g3, g4, pc, prev_action, allow_motion):
    ad = ar = al = au = 1
    wl = manhattanweights(g1, g2, g3, g4, pc)
    gl = [g1, g2, g3, g4]
    # weighted sum of ghosts position in each direction
    # higher sum, indicates ghosts closer to pacman in respective direction
    for g, w in zip(gl, wl):
        if g[0] > pc[0]:
            al += w / (g[0] - pc[0] + 1e-7)
        elif g[0] < pc[0]:
            ar += w / (pc[0] - g[0] + 1e-7)
        if g[1] > pc[1]:
            au += w / (g[1] - pc[1] + 1e-7)
        elif g[1] < pc[1]:
            ad += w / (pc[1] - g[1] + 1e-7)

    # Inverse to obtain higher values indicate more probable direction of motion
    ad = 1 / ad
    au = 1 / au
    ar = 1 / ar
    al = 1 / al

    # Likely to repeat action, so scale up probability
    if prev_action == "r":
        ar = ar * 1.25
    elif prev_action == "l":
        al = al * 1.25
    elif prev_action == "u":
        au = au * 1.25
    elif prev_action == "d":
        ad = ad * 1.25

    # normalize probabilities
    sum = al + ad + au + ar
    p_l = al / sum
    p_r = ar / sum
    p_u = au / sum
    p_d = ad / sum

    # Given Dictionary of allowable motions, if cannot move direction, split the probabilities to other directions
    if allow_motion["u"] == 0:
        p_r += 0.4 * p_u
        p_l += 0.4 * p_u
        p_d += 0.2 * p_u
        p_u = 0

    if allow_motion["d"] == 0:
        p_r += 0.4 * p_d
        p_l += 0.4 * p_d
        p_u += 0.2 * p_u
        p_d = 0

    if allow_motion["r"] == 0:
        p_u += 0.4 * p_r
        p_d += 0.4 * p_r
        p_l += 0.2 * p_r
        p_r = 0

    if allow_motion["l"] == 0:
        p_u += 0.4 * p_l
        p_d += 0.4 * p_l
        p_r += 0.2 * p_l
        p_l = 0

    return p_l, p_r, p_u, p_d

In [175]:
class Extract:
    """
    Extracts all relevant information from the current frame
    """

    def __init__(self):
        self.object_colors = {
            "pacman": {"lower": (25, 100, 100), "upper": (35, 255, 255)},  # Yellow
            "red_ghost": [
        {"lower": (0, 100, 100), "upper": (5, 255, 255)},   # Red range 1
        {"lower": (160, 100, 100), "upper": (180, 255, 255)}  # Red range 2
    ],  # Red
            "cyan_ghost": {"lower": (80, 100, 100), "upper": (100, 255, 255)},  # Cyan
            "pink_ghost": {"lower": (110, 20, 50), "upper": (160, 110, 255)},  # Pink
            "orange_ghost": {"lower": (5, 20, 100), "upper": (20, 255, 255)},  # Orange
        }
        self.ghost_marker_colors = {
            "pacman": (0, 255, 255),
            "red_ghost": (0, 0, 255),
            "cyan_ghost": (255, 255, 0),
            "pink_ghost": (255, 0, 255),
            "orange_ghost": (0, 100, 255),
        }
        self.movements = {
            1: "up",
            -1: "down",
            -10: "left",
            10: "right",
            11: "upright",
            -11: "downleft",
            9: "upleft",
            -9: "downright",
            0: "nomo",
        }

    # def find_center(self, mask: cv2.Mat):
    #     """
    #     Find the center of the object in the mask

    #     Args:
    #         mask: binary mask of the object

    #     Returns:
    #         (cx, cy): center of the object
    #     """
    #     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #     if contours:
    #         largest_contour = max(contours, key=cv2.contourArea)
    #         M = cv2.moments(largest_contour)
    #         if M["m00"] != 0:
    #             cx = int(M["m10"] / M["m00"])
    #             cy = int(M["m01"] / M["m00"])
    #             return (cx, cy)
    #     return None

    def find_all_centers(self, mask: cv2.Mat):
        """
        Find all centers of the object in the mask

        Args:
            mask: binary mask of the object

        Returns:
            centers: list of centers of the object
        """

        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        centers = []
        for contour in contours:
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                
                # Calculate area and perimeter
                area = M["m00"]
                perimeter = cv2.arcLength(contour, True)
                
                # Calculate the aspect ratio of the bounding rectangle
                x, y, w, h = cv2.boundingRect(contour)
                if w > h:
                    aspect_ratio = float(w) / h
                else:
                    aspect_ratio = float(h) / w
                
                # Calculate the extent
                rect_area = w * h
                extent = float(area) / rect_area
                
                # Calculate the convex hull area and solidity
                hull = cv2.convexHull(contour)
                hull_area = cv2.contourArea(hull)
                solidity = float(area) / hull_area
                
                # Filter based on circular properties
                if area > 250 :  # Adjust thresholds as needed
                    centers.append((cx, cy))
        return centers

    def bgr_to_hsv(self, frame: cv2.Mat):
        """converts BGR image to HSV image"""
        return cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    def create_combined_mask(self, hsv_frame: cv2.Mat, color_ranges):
        """
        Creates a combined mask from multiple color ranges.

        Args:
            hsv_frame: current frame in HSV.
            color_ranges: list or dict of lower and upper HSV bounds.

        Returns:
            combined_mask: a binary mask combining all ranges.
        """
        combined_mask = None
        if isinstance(color_ranges, list):
            for color_range in color_ranges:
                mask = cv2.inRange(hsv_frame, np.array(color_range["lower"]), np.array(color_range["upper"]))
                combined_mask = mask if combined_mask is None else cv2.bitwise_or(combined_mask, mask)
        else:
            combined_mask = cv2.inRange(hsv_frame, np.array(color_ranges["lower"]), np.array(color_ranges["upper"]))
        return combined_mask

    def extract_locations(self, frame: cv2.Mat, multi=False, remove_spawn_point=True):
        """
        Extracts the locations of all objects in the frame.

        Args:
            frame: current frame in BGR.
            multi: whether to extract multiple objects of the same type.
            remove_spawn_point: whether to remove the spawn point from the frame.

        Returns:
            positions: dictionary containing the positions of all objects.
        """
        positions = {}
        if remove_spawn_point:
            frame[220:240, 170:190, :] = 0
        hsv = self.bgr_to_hsv(frame)

        for name, color_ranges in self.object_colors.items():
            combined_mask = self.create_combined_mask(hsv, color_ranges)
            center = self.find_all_centers(combined_mask) if multi else self.find_center(combined_mask)
            positions[name] = center

        return positions

    def extract_movement(self, positions: dict, prev_positions: dict):
        """
        Extracts the movement of all objects in the frame

        Args:
            frame: current frame in BGR
            prev_positions: dictionary containing the positions of all objects in the previous frame

        Returns:
            movements: dictionary containing the movements of all objects
        """
        movements = {}
        for name in positions:
            if name in prev_positions:
                prev_position = prev_positions[name]
                if prev_position is not None and positions[name] is not None:
                    movements[name] = self.movements[
                        np.sign(positions[name][0] - prev_position[0]) * 10
                        + np.sign(positions[name][1] - prev_position[1])
                    ]
                else:
                    movements[name] = None
            else:
                movements[name] = None
        return movements

    def valid_entity_movements(self, frame: cv2.Mat, entity_center: tuple, offset=20):
        """
        Checks for walls in the immediate vicinity of Pac-Man (up, down, left, right).

        Args:
            frame: Current frame in BGR
            pacman_center: (cx, cy) center of Pac-Man
            offset: How many pixels away from the center to check for a wall

        Returns:
            A dictionary indicating whether pacman can move in each direction.
            e.g. {"up": True, "down": False, "left": True, "right": False}
        """
        try:
            # If we have no detected Pac-Man, just return False in all directions
            if entity_center is None:
                return {"up": False, "down": False, "left": False, "right": False}

            hsv = self.bgr_to_hsv(frame)

            # Define a rough HSV range for the blue walls (you may need to adjust these)
            walls_lower = (100, 100, 100)
            walls_upper = (130, 255, 255)

            # Create a mask that highlights the walls
            walls_mask = cv2.inRange(hsv, walls_lower, walls_upper)

            cx, cy = entity_center
            height, width = walls_mask.shape

            # Safeguard boundaries
            up_y = max(cy - offset, 0)
            down_y = min(cy + offset, height - 1)
            left_x = max(cx - offset, 0)
            right_x = min(cx + offset, width - 1)

            # Check pixel values in rectangle in each direction

            up_wall_matrix = walls_mask[up_y : cy - 10, cx - 10 : cx + 10]
            down_wall_matrix = walls_mask[cy + 10 : down_y, cx - 10 : cx + 10]
            left_wall_matrix = walls_mask[cy - 10 : cy + 10, left_x : cx - 10]
            right_wall_matrix = walls_mask[cy - 10 : cy + 10, cx + 10 : right_x]

            up_wall = ~np.any(up_wall_matrix)
            down_wall = ~np.any(down_wall_matrix)
            left_wall = ~np.any(left_wall_matrix)
            right_wall = ~np.any(right_wall_matrix)

            return {
                "up": up_wall,
                "down": down_wall,
                "left": left_wall,
                "right": right_wall,
            }
        except Exception as e:
            print(e)
            print(entity_center)
            return {"up": False, "down": False, "left": False, "right": False}
        


In [176]:
################################################################################
# 1. Use your existing code (Extract, motion_model, manhattanweights, etc.)
################################################################################

# For illustration, we assume we have these available:
#   class Extract:
#       ...
#   def manhattanweights(g1, g2, g3, g4, pc):
#       ...
#   def motion_model(g1, g2, g3, g4, pc, prev_action, allow_motion):
#       ...

################################################################################
# 2. Define a ParticleFilter for Pac-Man
################################################################################


class ParticleFilter:
    def __init__(
        self,
        num_particles,
        frame_width,
        frame_height,
        extract_instance,
        spawn_point=(190, 315),
        init_prev_action="r",
        init_method="random",
        resample_method="multinomial",
        motion_noise_std=3.0,  # <--- Gaussian noise for motion
        measurement_noise_px=2,  # <--- Pixel noise for measurements
    ):
        """
        Particle Filter for tracking Pac-Man

        Args:
            num_particles (int): Number of particles
            frame_width (int): Width of the game/maze frame
            frame_height (int): Height of the game/maze frame
            extract_instance (Extract): An instance of your Extract class
            init_prev_action (str): An initial guess for Pac-Man's previous action
            init_method (str): Method to initialize particles ('random' or 'center')
            motion_noise_std (float): Standard deviation of the Gaussian noise added
                                      to each motion step
            measurement_noise_px (int): Max random integer offset added to each
                                        measured detection location
        """
        self.num_particles = num_particles
        self.frame_width = frame_width
        self.frame_height = frame_height
        self.extract = extract_instance
        self.init_prev_action = init_prev_action
        self.init_method = init_method
        self.spawn_point = spawn_point
        self.resample_method = resample_method
        self.motion_noise_std = motion_noise_std
        self.measurement_noise_px = measurement_noise_px

        # Each particle: (x, y, prev_action)
        self.entities = ["pacman", "red_ghost", "cyan_ghost", "pink_ghost", "orange_ghost"]
        self.particles = {entity: [] for entity in self.entities}
        self.weights = {entity: np.ones(num_particles, dtype=np.float32) / num_particles for entity in self.entities}

        for entity in self.entities:
            self.initialize_particles(entity)

    def initialize_particles(self,entity):
        """Initialize particles in the valid region of the frame."""
        self.particles[entity] = []
        for _ in range(self.num_particles):
            if self.init_method == "random":
                x = random.randint(0, self.frame_width - 1)
                y = random.randint(0, self.frame_height - 1)
            elif self.init_method == "center":
                x = self.frame_width // 2
                y = self.frame_height // 2
            elif self.init_method == "spawn_point" and entity == "pacman":
                if random.random() < 0.5:
                    x = random.randint(
                        self.spawn_point[0] - 40, self.spawn_point[0] + 50
                    )
                    y = random.randint(
                        self.spawn_point[1] - 25, self.spawn_point[1] + 25
                    )
                else:
                    x = random.randint(0, self.frame_width - 1)
                    y = random.randint(0, self.frame_height - 1)
            else:
                x = random.randint(0, self.frame_width - 1)
                y = random.randint(0, self.frame_height - 1)

            self.particles[entity].append((x, y, self.init_prev_action))

        self.particles[entity] = np.array(self.particles[entity], dtype=object)
    
    def delete_ghosts(self,entity):
        self.particles[entity] = None


    def step(self, frame, prev_frame_positions):
        """
        Main PF step for one iteration.

        1) Extract ghost positions from the current frame
        2) Determine allowed movements for Pac-Man (walls, etc.)
        3) Motion update: apply motion_model to each particle + motion noise
        4) Observation update: weigh each particle by how well it matches the detection
           (with measurement noise)
        5) Resample
        6) Estimate the best Pac-Man position from the particles

        Args:
            frame (np.ndarray): Current game frame (BGR)
            prev_frame_positions (dict): Positions from the previous frame
                                         (e.g. from extract_locations)

        Returns:
            best_estimate (tuple): A guess of Pac-Man's position (x, y)
        """
        # 1) Extract ghost positions
        # curr_positions = self.extract.extract_locations(
        #     frame, multi=False, remove_spawn_point=True
        # )

        ghost_keys = ["red_ghost", "cyan_ghost", "pink_ghost", "orange_ghost"]
        # Some ghost positions might be None if not detected
        # ghosts = []
        # for gkey in ghost_keys:
        #     pos = curr_positions.get(gkey, None)
        #     if pos is None:
        #         self.delete_ghosts(gkey)
        #         # If not detected, let's just put something far away
        #         ghosts.append((9999, 9999))
        #     else:
        #         if self.particles[gkey] is None:
        #             self.initialize_particles(gkey)
        #         ghosts.append(pos)

        
        # 2) Determine if Pac-Man can move in each direction (allowable moves).
        #    For demonstration, we'll get the "best guess" of Pac-Man from the last step
        #    or from the previous frame positions if available.
        if prev_frame_positions and prev_frame_positions.get("pacman") is not None:
            # Might be a list of centers or a single center
            prev_pacman_pos = prev_frame_positions["pacman"]
            if len(prev_pacman_pos) == 0:
                px = int(np.mean(self.particles["pacman"][:, 0]))
                py = int(np.mean(self.particles["pacman"][:, 1]))
                best_guess_pacman = (px, py)
            elif isinstance(prev_pacman_pos, list) and len(prev_pacman_pos) > 0:
                best_guess_pacman = prev_pacman_pos[0]
            else:
                best_guess_pacman = prev_pacman_pos
        else:
            # If no info, just pick the average of the current particles
            px = int(np.mean(self.particles["pacman"][:, 0]))
            py = int(np.mean(self.particles["pacman"][:, 1]))
            best_guess_pacman = (px, py)
        ghostspos = {}
        for gkey in ghost_keys:
            if prev_frame_positions and prev_frame_positions.get(gkey) is not None and self.particles[gkey] is not None:
                # Might be a list of centers or a single center
                prev_ghost_pos = prev_frame_positions[gkey]
                if prev_ghost_pos is not None and isinstance(prev_ghost_pos, list) and len(prev_ghost_pos) > 0:
                    bestgpos = prev_ghost_pos[0]
                else:
                    bestgpos = prev_ghost_pos
            else:
                # If no info, just pick the average of the current particles
                if self.particles[gkey] is not None:
                    px = int(np.mean(self.particles[gkey][:, 0]))
                    py = int(np.mean(self.particles[gkey][:, 1]))
                    bestgpos = (px, py)
                else:
                    bestgpos = None
                    
            ghostspos[gkey] = bestgpos


        allow_dict = self._compute_allow_dict(frame, best_guess_pacman)
        ghostallow = {}
        for gkey in ghost_keys:
            if self.particles[gkey] is not None:
                ghostallow[gkey] = self._compute_allow_dict(frame, ghostspos[gkey])
            
        ghosts = []
        for gkey in ghost_keys:
            if self.particles[gkey] is None:
                # If not detected, let's just put something far away
                ghosts.append((9999, 9999))
            else:
                ghosts.append(ghostspos[gkey])
        # 3) Motion update
        self._motion_update(ghosts, allow_dict)
        for gkey in ghost_keys:
            if self.particles[gkey] is not None:
                self.ghost_motion(gkey,best_guess_pacman,ghostallow[gkey])

        # 4) Observation update
        positions = {}
        for entity in self.entities:
            positions[entity] = self._observation_update(frame,entity)
            if positions[entity] is None:
                print("Entity ",entity," None ")
            if entity in ghost_keys:
                if positions[entity] is None:
                    self.delete_ghosts(entity)
                else:
                    if self.particles[entity] is None:
                        self.initialize_particles(entity)

        # 5) Resample
        for entity in self.entities:
            if self.particles[entity] is not None:
                self._resample(entity,method=self.resample_method)

        # 6) Estimate

        best_estimate = {}
        for entity in self.entities:
            if self.particles[entity] is not None:
                best_estimate[entity] = self.estimate(entity)
            else:
                best_estimate[entity] = None
        return best_estimate, positions

    def ghost_motion(self,ghost,pc,allow_motion_dict):
        
        updated_particles = []
        for i, (x, y, prev_act) in enumerate(self.particles[ghost]):
            ghostposition = (x, y)
            # motion_model returns p_left, p_right, p_up, p_down
            p_l, p_r, p_u, p_d = motion_model_ghosts(
                ghostposition,pc,allow_motion_dict
            )

            # Normalize if necessary
            probs = np.array([p_l, p_r, p_u, p_d])
            probs_sum = probs.sum()
            if probs_sum > 0:
                probs /= probs_sum
            else:
                # fallback: uniform
                probs = np.ones(4) / 4.0
            action = np.random.choice(["l", "r", "u", "d"], p=probs)

            # Move the particle
            new_x, new_y = x, y
            if action == "l":
                new_x = x - 2
            elif action == "r":
                new_x = x + 2
            elif action == "u":
                new_y = y - 2
            elif action == "d":
                new_y = y + 2

            # Clamp bounds
            new_x = max(0, min(self.frame_width - 1, new_x))
            new_y = max(0, min(self.frame_height - 1, new_y))
            # new_x = new_x % self.frame_width
            # new_y = new_y % self.frame_height

            # Add motion noise (Gaussian).
            # We'll clamp again in case noise pushes it out of bounds.
            noise_dx = int(round(random.gauss(0, self.motion_noise_std)))
            noise_dy = int(round(random.gauss(0, self.motion_noise_std)))
            new_x = max(0, min(self.frame_width - 1, new_x + noise_dx))
            new_y = max(0, min(self.frame_height - 1, new_y + noise_dy))

            # new_x = (new_x + noise_dx) % self.frame_width
            # new_y = (new_y + noise_dy) % self.frame_height

            updated_particles.append((new_x, new_y, action))

        self.particles[ghost] = np.array(updated_particles, dtype=object)
        
    def _motion_update(self, ghosts, allow_motion_dict):
        """
        Moves each particle according to your 'motion_model'. We:
          - Extract the probabilities from motion_model(...)
          - Sample an action
          - Update particle state
          - Add random motion noise to x and/or y
        """
        updated_particles = []
        for i, (x, y, prev_act) in enumerate(self.particles["pacman"]):
            pc = (x, y)
            g1, g2, g3, g4 = ghosts
            # motion_model returns p_left, p_right, p_up, p_down
            p_l, p_r, p_u, p_d = motion_model(
                g1, g2, g3, g4, pc, prev_act, allow_motion_dict
            )

            # Normalize if necessary
            probs = np.array([p_l, p_r, p_u, p_d])
            probs_sum = probs.sum()
            if probs_sum > 0:
                probs /= probs_sum
            else:
                # fallback: uniform
                probs = np.ones(4) / 4.0

            action = np.random.choice(["l", "r", "u", "d"], p=probs)

            # Move the particle
            new_x, new_y = x, y
            if action == "l":
                new_x = x - 2
            elif action == "r":
                new_x = x + 2
            elif action == "u":
                new_y = y - 2
            elif action == "d":
                new_y = y + 2

            # Clamp bounds
            new_x = max(0, min(self.frame_width - 1, new_x))
            new_y = max(0, min(self.frame_height - 1, new_y))
            # new_x = new_x % self.frame_width
            # new_y = new_y % self.frame_height

            # Add motion noise (Gaussian).
            # We'll clamp again in case noise pushes it out of bounds.
            noise_dx = int(round(random.gauss(0, self.motion_noise_std)))
            noise_dy = int(round(random.gauss(0, self.motion_noise_std)))
            new_x = max(0, min(self.frame_width - 1, new_x + noise_dx))
            new_y = max(0, min(self.frame_height - 1, new_y + noise_dy))

            # new_x = (new_x + noise_dx) % self.frame_width
            # new_y = (new_y + noise_dy) % self.frame_height

            updated_particles.append((new_x, new_y, action))

        self.particles["pacman"] = np.array(updated_particles, dtype=object)

    def _observation_update(self, frame,entity):
        """
        Weigh each particle by how well it matches the detected positions of Pac-Man.
        If multiple centers are found, we split probability among them.

        Adds measurement noise by artificially jittering the positions we get from
        extract_locations. This simulates uncertain detection.
        """
        positions = self.extract.extract_locations(
            frame, multi=True, remove_spawn_point=False
        )
        entity_centers = positions[entity]

        # If no detection, you might want to reduce weights or keep them as is
        if not entity_centers:
            self.weights[entity] *= 0.8
            return
        
        if self.particles[entity] is None:
            self.weights[entity] *= 0.8
            return positions[entity]

        # Inject measurement noise into each detected center
        noisy_centers = []
        for c in entity_centers:
            # small random offset in x and y
            # ± self.measurement_noise_px
            nx = c[0] + random.randint(
                -self.measurement_noise_px, self.measurement_noise_px
            )
            ny = c[1] + random.randint(
                -self.measurement_noise_px, self.measurement_noise_px
            )
            # clamp
            nx = max(0, min(self.frame_width - 1, nx))
            ny = max(0, min(self.frame_height - 1, ny))
            noisy_centers.append((nx, ny))

        match_threshold = 15
        num_centers = len(noisy_centers)
        center_prob = 1.0 / num_centers

        for i, (x, y, _) in enumerate(self.particles[entity]):
            # Find distance to the closest noisy center
            min_dist = float("inf")
            for c in noisy_centers:
                dist = np.hypot(x - c[0], y - c[1])
                if dist < min_dist:
                    min_dist = dist

            if min_dist < match_threshold:
                # The closer, the higher the weighting. Example: 1 + fraction * (match_threshold - dist)/match_threshold
                self.weights[entity][i] *= (
                    1.0 + center_prob * (match_threshold - min_dist) / match_threshold
                )
            else:
                # If too far from all detections, weight is reduced
                self.weights[entity][i] *= 0.5

        # Normalize weights
        weight_sum = np.sum(self.weights[entity])
        if weight_sum > 0:
            self.weights[entity] /= weight_sum
        else:
            self.weights[entity][:] = 1.0 / len(self.weights[entity])

        return positions[entity]

    def _resample(self, entity,method="multinomial"):
        """
        Systematic or multinomial resampling of the particles
        based on current weights.
        """
        N = self.num_particles
        new_particles = []
        # normalized weights
        self.weights[entity] = self.weights[entity] / np.sum(self.weights[entity])

        if method == "multinomial":
            indices = np.random.choice(range(N), size=N, p=self.weights[entity])
            for idx in indices:
                new_particles.append(self.particles[entity][idx])

            self.particles[entity] = np.array(new_particles, dtype=object)
            self.weights[entity] = np.ones(N, dtype=np.float32) / N
        elif method == "systematic":
            indices = np.zeros(N, dtype=int)
            r = np.random.rand() / N
            c = self.weights[entity][0]
            i = 0
            for m in range(N - 1):
                U = r + m / N
                while U > c:
                    i += 1
                    c += self.weights[entity][i]
                indices[m] = i

            for idx in indices:
                new_particles.append(self.particles[entity][idx])

            self.particles[entity] = np.array(new_particles, dtype=object)
            self.weights[entity] = np.ones(N, dtype=np.float32) / N

    def estimate(self,entity):
        """
        Estimate Pac-Man's position from the particles (e.g. weighted mean).
        Returns:
            (est_x, est_y)
        """
        if self.weights[entity].sum() < 1e-7:
            # Fallback
            est_x = int(np.mean(self.particles[entity][:, 0]))
            est_y = int(np.mean(self.particles[entity][:, 1]))
            return (est_x, est_y)

        # Weighted average
        w_norm = self.weights[entity] / self.weights[entity].sum()
        est_x = int(np.sum(self.particles[entity][:, 0] * w_norm))
        est_y = int(np.sum(self.particles[entity][:, 1] * w_norm))
        return (est_x, est_y)

    def _compute_allow_dict(self, frame, entity_center):
        """
        Use your valid_pacman_movements(...) (slightly modified) to get a dictionary:
          {'u': 1/0, 'd': 1/0, 'l': 1/0, 'r': 1/0}
        """
        valid_moves = self.extract.valid_entity_movements(frame, entity_center)
        allow_dict = {
            "u": 1 if valid_moves["up"] else 0,
            "d": 1 if valid_moves["down"] else 0,
            "l": 1 if valid_moves["left"] else 0,
            "r": 1 if valid_moves["right"] else 0,
        }
        return allow_dict

In [177]:
def main_loop():
    # Instantiate your Extract
    extract = Extract()  # from your code

    # Example: open a webcam or video
    cap = cv2.VideoCapture("pacmanvid.mp4")

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_filename = "random_init.avi"  # Output video file name
    frame_width, frame_height = frame_width, frame_height  # Frame size
    fps = 10  # Frames per second
    fps = 30  # Frames per second

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"XVID")  # Codec
    out = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))

    # Create the PF, now with noise parameters
    pf = ParticleFilter(
        num_particles=500,
        frame_width=frame_width,
        frame_height=frame_height,
        extract_instance=extract,
        init_prev_action="r",
        init_method="random",
        resample_method="multinomial",
        motion_noise_std=4,  # tweak to suit your environment
        measurement_noise_px=3,  # tweak to suit your environment
    )


    # Define colors for each entity
    colors = {
        "pacman": (0, 255, 255),       # Yellow
        "red_ghost": (0, 0, 255),      # Red
        "cyan_ghost": (255, 255, 0),   # Cyan
        "pink_ghost": (255, 0, 0),   # Pink
        "orange_ghost": (0, 165, 255)  # Orange
    }

    prev_positions = None
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        best_estimates, positions = pf.step(frame, prev_positions)

        # For debug: draw particles for each entity
        for entity in pf.entities:
            if best_estimates[entity] is not None:
                entity_particles = pf.particles[entity]
                for x, y, _ in entity_particles:
                    cv2.circle(frame, (x, y), 1, colors[entity], -1)

                # Draw the best estimate in a larger circle
                best_estimate = best_estimates[entity]
                cv2.circle(frame, best_estimate, 5, colors[entity], -1)

                if positions and positions.get(entity, None):
                    if isinstance(positions[entity], list):
                        for c in positions[entity]:
                            cv2.circle(frame, c, 5, colors[entity], 2)
                    elif positions[entity] is not None:
                        cv2.circle(frame, positions[entity], 5, colors[entity], 2)

        # Update prev_positions for next iteration if needed
        prev_positions = positions

        cv2.imshow("Pacman Tracking PF (with noise)", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break  # Esc to quit
        # print("Best Estimate Locations:")
        # for entity in pf.entities:
        #     print(f"{entity.capitalize()}: {best_estimates[entity]}")
        #     if positions and positions.get(entity, None):
        #         print(f"Detected {entity.capitalize()} Locations: {positions[entity]}")

        out.write(frame)

    out.release()
    cap.release()
    cv2.destroyAllWindows()



In [178]:
main_loop()

Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 
Entity  cyan_ghost  None 


In [179]:
extract = Extract()  # from your code

# Example: open a webcam or video
cap = cv2.VideoCapture("4m.mp4")

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create the PF, now with noise parameters
pf = ParticleFilter(
    num_particles=2000,
    frame_width=frame_width,
    frame_height=frame_height,
    extract_instance=extract,
    init_prev_action="r",
    init_method="spawn_point",
    motion_noise_std=2,  # tweak to suit your environment
    measurement_noise_px=3,  # tweak to suit your environment
)

In [180]:
cap = cv2.VideoCapture("4m.mp4")
frame_count = 0
frames = []
while True:
    ret, frame = cap.read()
    frame_count += 1
    frames.append(frame)
    if frame_count > 10:
        break

In [181]:
frame_iter = iter(frames)

In [182]:
len(frames)

11

In [183]:
positions = {"pacman": pf.spawn_point}

In [184]:
frame = next(frame_iter)
best_estimate = pf.step(frame, prev_frame_positions=positions)
positions = extract.extract_locations(frame, multi=True, remove_spawn_point=False)
for particle in pf.particles:
    cv2.circle(frame, (particle[0], particle[1]), 1, (0, 255, 0), -1)
cv2.circle(frame, best_estimate, 10, (0, 0, 255), -1)
cv2.imshow("Pacman Tracking PF (with noise)", frame)
while True:
    k = cv2.waitKey(0)
    if k == 27:
        break

error: OpenCV(4.10.0) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type
>  - Can't parse 'center'. Sequence item with index 0 has a wrong type


In [109]:
positions

{'pacman': [(190, 315),
  (188, 240),
  (179, 240),
  (224, 233),
  (210, 235),
  (197, 236),
  (168, 236),
  (156, 236)],
 'red_ghost': [(196, 166),
  (182, 166),
  (189, 164),
  (178, 162),
  (182, 161),
  (196, 153)],
 'cyan_ghost': [(162, 198)],
 'pink_ghost': [(279, 369),
  (189, 344),
  (99, 369),
  (315, 310),
  (249, 296),
  (129, 296),
  (63, 310),
  (189, 264),
  (269, 236),
  (109, 236),
  (217, 335),
  (221, 189),
  (210, 188),
  (188, 198),
  (156, 188),
  (188, 185),
  (190, 196),
  (191, 155),
  (181, 155),
  (262, 137),
  (322, 96),
  (189, 104),
  (116, 136),
  (56, 97),
  (322, 50),
  (249, 50),
  (129, 49),
  (56, 50),
  (195, 62)],
 'orange_ghost': [(216, 200),
  (216, 186),
  (196, 166),
  (182, 166),
  (189, 164),
  (178, 162),
  (182, 161),
  (196, 153)]}

In [115]:
frame = next(frame_iter)
best_estimate = pf.step(frame, prev_frame_positions=positions)
positions = extract.extract_locations(frame, multi=True, remove_spawn_point=True)
for particle in pf.particles:
    cv2.circle(frame, (particle[0], particle[1]), 1, (0, 255, 0), -1)
cv2.circle(frame, best_estimate, 10, (0, 0, 255), -1)
cv2.imshow("Pacman Tracking PF (with noise) step 2", frame)

while True:
    k = cv2.waitKey(0)
    if k == 27:
        break

In [113]:
cv2.destroyAllWindows()

In [49]:
cap = cv2.VideoCapture("4m.mp4")
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [66]:
main_loop()

In [46]:
cv2.destroyAllWindows()

In [24]:
extract = Extract()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Extract the positions of all objects in the frame
    positions = extract.extract_locations(frame, multi=False, remove_spawn_point=False)

    # for name, value in positions.items():
    #     if value is not None:
    #         cv2.circle(frame, value, 5, extract.ghost_marker_colors[name], 10)
    cv2.circle(
        frame, positions["pacman"], 20, extract.ghost_marker_colors["pacman"], -1
    )

    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
N = 2000
new_particles = []

# random particles
particles = np.random.randint(0, 300, (N, 2))
# random weights
weights = np.random.rand(N)

weights = weights / np.sum(weights)
indices = np.random.choice(range(N), size=N, p=weights)
for idx in indices:
    new_particles.append(particles[idx])

particles = np.array(new_particles, dtype=object)
weights = np.ones(N, dtype=np.float32) / N


In [12]:
len(indices)

2000